# Pasos a seguir

## Pasos para realizar el reto de extracción de información de facturas eléctricas

**1. Recopilación de recursos:**

* Asegúrate de tener instalado Python en tu ordenador.
* Descarga el dataset de entrenamiento que contiene las facturas PDF y sus correspondientes archivos JSON.
* Investiga y selecciona las librerías de Python adecuadas para:
    * Extraer texto de archivos PDF: Considera librerías como `PyPDF2`, `Poppler` o `pdfminer.six`.
    * Procesamiento de texto: Librerías como `re`, `string` o `nltk` pueden ser útiles para la limpieza y manipulación del texto extraído.
    * Aprendizaje automático: Si decides utilizar un modelo de aprendizaje automático para la extracción de información, necesitarás librerías como `scikit-learn` o `TensorFlow`.

**2. Preprocesamiento de datos:**

* Lee los archivos PDF y JSON del dataset de entrenamiento.
* Limpia el texto extraído de los PDF, eliminando caracteres irrelevantes, convirtiendo todo a minúsculas y manejando correctamente los caracteres con tilde.
* Estructura los datos de manera que se facilite su procesamiento, por ejemplo, creando un diccionario para cada factura que contenga los campos extraídos del PDF y su correspondiente valor del archivo JSON.

**3. Exploración y análisis de datos:**

* Analiza la estructura y el contenido de las facturas PDF y los archivos JSON para identificar patrones y características comunes.
* Identifica posibles variaciones en el formato y la disposición de la información dentro de las facturas.
* Segmenta los datos en diferentes grupos según sus características, por ejemplo, por tipo de comercializadora o formato de factura.

**4. Definición de la estrategia de extracción:**

* Decide si utilizar un enfoque basado en reglas o un modelo de aprendizaje automático para la extracción de información.
* **Enfoque basado en reglas:**
    * Define reglas manuales para identificar y extraer cada campo de información en base a su ubicación, formato y características dentro del PDF.
    * Considera el uso de expresiones regulares o técnicas de procesamiento del lenguaje natural para la extracción precisa de la información.
* **Enfoque basado en aprendizaje automático:**
    * Elige un algoritmo de aprendizaje automático adecuado para la tarea de extracción de información, como clasificación o aprendizaje supervisado.
    * Entrena el modelo utilizando los datos preprocesados del dataset de entrenamiento.
    * Evalúa el rendimiento del modelo en un conjunto de datos de validación para asegurar su generalización a nuevas facturas.

**5. Implementación de la solución:**

* Implementa el algoritmo de extracción de información, ya sea basado en reglas o en aprendizaje automático.
* Desarrolla la lógica para procesar cada factura PDF, extraer la información relevante y almacenarla en un formato estructurado.
* Maneja posibles casos de error o situaciones donde la información no se encuentre en el formato esperado.

**6. Evaluación y mejora:**

* Evalúa el rendimiento de la solución utilizando el script proporcionado para obtener el score de Levenshtein.
* Analiza los resultados obtenidos para identificar áreas de mejora y posibles errores en la extracción de información.
* Refina la estrategia de extracción, las reglas o el modelo de aprendizaje automático en base a los resultados de la evaluación.
* Repite el proceso de evaluación y mejora hasta obtener un score satisfactorio.

**7. Consideraciones adicionales:**

* Es importante tener en cuenta la diversidad de formatos y estructuras de las facturas para garantizar la generalización del método de extracción.
* La utilización de técnicas de procesamiento del lenguaje natural puede ser útil para mejorar la precisión de la extracción, especialmente en casos de ambigüedad o falta de información.
* Es recomendable implementar mecanismos para manejar errores y casos excepcionales, como la ausencia de campos o formatos de datos inesperados.
* La documentación del código y la estrategia de extracción es fundamental para facilitar su comprensión y mantenimiento.

Recuerda que este es un reto complejo que requiere habilidades en programación, procesamiento de texto, aprendizaje automático y análisis de datos. Es importante dedicar tiempo a la comprensión del problema, la exploración de los datos y la experimentación con diferentes enfoques para encontrar la solución más adecuada.


# Librerías

Para realizar la limpieza del texto extraído de los PDFs, te recomiendo utilizar las siguientes librerías de Python:

**re:** Módulo para expresiones regulares, que te permitirá identificar y eliminar patrones de caracteres irrelevantes.

**string:** Módulo que contiene funciones útiles para el manejo de cadenas de texto, como la conversión a minúsculas.

**unicodedata:** Módulo que proporciona funciones para trabajar con caracteres Unicode, incluyendo la normalización de caracteres con tilde.

In [1]:
import pandas as pd
import numpy as np

# Leer pdf
import PyPDF2
#import pdfplumber
import fitz  # PyMuPDF
#from pdfminer.high_level import extract_text
import PyPDF4
import slate3k as slate

# Procesamiento texto
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import unicodedata

# Guardar archivos
import pickle

# Base de datos sql
import mysql.connector

# Buscar provincia a través de código postal
import pgeocode

# Datos a extraer:

- Nombre del cliente (nombre_cliente) **HECHO**
- DNI del cliente (dni_cliente) **HECHO**
- Calle del clilente (calle_cliente) **HECHO**
- Código postal del cliente (cp_cliente) **HECHO**
- Población del cliente (población_cliente) **HECHO**
- Provincia del cliente (provincia_cliente) **HECHO**
- Nombre de la empresa comercializadora (nombre_comercializadora)
- CIF de la comercializadora (cif_comercializadora) **HECHO MENOS FACTURA 631**
- Dirección de la comercializadora (dirección_comercializadora)
- Código postal de la comercializadora (cp_comercializadora)
- Población de la comercializadora (población_comercializadora)
- Provincia de la comercializadora (provincia_comercializadora)
- Número de factura (nümero_factura)
- Inicio del periodo de facturación (inicio_periodo)
- Fin del periodo de facturacón (fin_periodo)
- Importe de la factura (importe_factura)
- Fecha del cargo (fecha_cargo)
- Consumo en el periodo (consumo_periodo)
- Potencia contratada (potencia_contratada)

### Leer los archivos

In [ ]:
# Abre el archivo PDF en modo lectura binaria
invoices = {}
errors = {}
count = 0

for i in range(1000):
    try:
        with open(f'./training/factura_{i}.pdf', 'rb') as file:
            print(f'./training/factura_{i}.pdf')
            reader = PyPDF2.PdfReader(file)
            # Obtén el número de páginas
            num_pages = len(reader.pages)
            content = {}
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                text = page.extract_text()
                content[page_num] = text
            invoices[i] = content
    except:
        print("Error al leer la factura")
        errors[count] = f"factura_{i}"
        count += 1
    print(f"{i}")

In [ ]:
# Función para leer una factura:
name = "nombre archivo sin extension"

def ft_readinvoice(name):
    invoices = {}
    errors = {}
    count = 0
    try:
        with open(f"./{name}.pdf", 'r') as file:
            print('name')
            reader = PyPDF2.PdfReader(file)
            # Obtén el número de páginas
            num_pages = len(reader.pages)
            content = {}
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                text = page.extract_text()
                content[page_num] = text
            invoices[0] = content
        with open(f"{name}.pkl", 'wb') as file:
            pickle.dump(invoices, file)
    except:
        print("""Error al leer la factura. 
            Comprueba que el archivo esté en el mismo directorio, 
            que el nombre del archivo sea válido (sin la extensión) 
            y que el archivo sea formato pdf""")
        errors[count] = name
        count += 1
        
    return invoices

In [4]:
# Guardar el diccionario en un archivo
with open('facturas.pkl', 'wb') as file:
    pickle.dump(invoices, file)

# Procesamiento de datos

### Funciones necesarias

* **Eliminar caracteres irrelevantes:** Utiliza expresiones regulares para eliminar caracteres como tabuladores, retornos de carro, caracteres de control y cualquier otro símbolo que no sea relevante para la extracción de información.
* **Convertir a minúsculas:** Convierte todo el texto a minúsculas utilizando la función `lower()` del módulo `string`.
* **Manejar caracteres con tilde:** Normaliza los caracteres con tilde utilizando la función `normalize()` del módulo `unicodedata` con el argumento `'NFKD'`. Esto descompone los caracteres con tilde en dos caracteres: la letra base y el acento.

In [80]:
# Función normalización de factura
def ft_cleaner(text):
    # Eliminar saltos de línea y tabuladores
    text = re.sub(r'[\n\t]', ' ', text)
    
    # Normalizar el texto (NFKD) y eliminar diacríticos (acentos)
    normalize_text = unicodedata.normalize('NFKD', text)
    text_tilde = ''.join(c for c in normalize_text if not unicodedata.combining(c))
    
    # Eliminar caracteres especiales adicionales, dejando solo letras, dígitos, espacios, guiones, comillas simples y comas
    final_text = re.sub(r"[^A-Za-z0-9\s\',/.]", '', text_tilde)
    
    # Quitar espacios adicionales
    final_text = ' '.join(final_text.split())

    final_text = final_text.lower()
    
    return final_text

In [81]:
with open('facturas.pkl', 'rb') as file:
    invoices = pickle.load(file)

In [82]:
clean_invoices = {}

for tittle, pages in invoices.items():
    clean_pages = {}
    for num_pages, content in pages.items():
        clean_pages[num_pages] = ft_cleaner(content)
        #clean_pages[num_pages] = unicodedata.normalize('NFKD', content)
    clean_invoices[tittle] = clean_pages

In [83]:
# Guardar el diccionario en un archivo
with open('facturas_limpias.pkl', 'wb') as file:
    pickle.dump(clean_invoices, file)

In [20]:
# Función para obtener factura normalizada

def ft_normalize(name):
    
    pickle_file = f"{name}.pkl"
    with open(pickle_file, 'rb') as file:
        invoices = pickle.load(file)
        
    clean_invoices = {}

    for tittle, pages in invoices.items():
        clean_pages = {}
        for num_pages, content in pages.items():
            clean_pages[num_pages] = ft_cleaner(content)
            #clean_pages[num_pages] = unicodedata.normalize('NFKD', content)
        clean_invoices[tittle] = clean_pages

    # Guardar el diccionario en un archivo
    with open(f'{name}_clean.pkl', 'wb') as file:
        pickle.dump(clean_invoices, file)
    
    return clean_invoices

# Extracción:

- Nombre del cliente (nombre_cliente) **HECHO**
- DNI del cliente (dni_cliente) **HECHO**
- Calle del clilente (calle_cliente) **HECHO**
- Código postal del cliente (cp_cliente) **HECHO**
- Población del cliente (población_cliente) **HECHO**
- Provincia del cliente (provincia_cliente) **HECHO**
- Nombre de la empresa comercializadora (nombre_comercializadora) **HECHO MENOS 46**
- CIF de la comercializadora (cif_comercializadora) **HECHO MENOS FACTURA 631**
- Dirección de la comercializadora (dirección_comercializadora) **HECHO MENOS 32**
- Código postal de la comercializadora (cp_comercializadora) **HECHO MENOS 48**
- Población de la comercializadora (población_comercializadora) **HECHO MENOS 30**
- Provincia de la comercializadora (provincia_comercializadora) **HECHO**
- Número de factura (nümero_factura)
- Inicio del periodo de facturación (inicio_periodo)
- Fin del periodo de facturacón (fin_periodo)
- Importe de la factura (importe_factura)
- Fecha del cargo (fecha_cargo)
- Consumo en el periodo (consumo_periodo)
- Potencia contratada (potencia_contratada)

In [2]:
with open('facturas_limpias.pkl', 'rb') as file:
    clean_invoices = pickle.load(file)

In [3]:
def encontrar_provincia_por_cp(codigo_postal, pais='ES'):
    # Crear un objeto Nominatim para el país especificado (por defecto España)
    nomi = pgeocode.Nominatim(pais)
    
    # Obtener la información geográfica del código postal
    info = nomi.query_postal_code(codigo_postal)
    
    # Verificar si se encontró información válida
    if info.county_name:
        return info.county_name
    else:
        return "none"

#### Función Nombre / Titular del Contrato / Titular

In [302]:
def ft_font_code(invoice, name):

    result = {}

    empty = {"dni_cliente": [], 
            "nombre_cliente": [], 
            "calle_cliente": [], 
            "población_cliente": [],
            "provincia_cliente": [],
            "cp_cliente": [],
            "nombre_comercializadora": [],
            "cif_comercializadora": [],
            "dirección_comercializadora": [],
            "cp_comercializadora": [],
            "provincia_comercializadora": [],
            "localidad_comercializadora": [],
            }

    filler = {"dni_cliente": [], 
            "nombre_cliente": [], 
            "calle_cliente": [], 
            "población_cliente": [],
            "provincia_cliente": [],
            "cp_cliente": [],
            "nombre_comercializadora": [],
            "cif_comercializadora": [],
            "dirección_comercializadora": [],
            "cp_comercializadora": [],
            "provincia_comercializadora": [],
            "localidad_comercializadora": [],
            }

    invoices = {}
    client_name = []
    dni = []
    adress_street = []
    adress_city = []
    adress_province = []
    adress_cp = []
    distributor_name = []
    cif = []
    distributor_city = []
    distributor_street = []
    distributor_province = []
    distributor_cp = []

    pages = invoice

    for n_pages, content in pages.items():
    
        # -----------------------------------------------------------------------------------------------------------
        # Client name
        patron_cn_1 = r'nombre (.*?) direccion'
        matches_1 = re.findall(patron_cn_1, content, re.IGNORECASE)
        for match in matches_1:
            if match:
                client_name.extend(matches_1)
        
        patron_cn_2 = r'titular del contrato (.*?) nif'
        matches_2 = re.findall(patron_cn_2, content, re.IGNORECASE)
        for match in matches_2:
            if match not in client_name:  # Solo agregar si no está en matches_1
                client_name.append(match)
        patron_cn_3 = r'titular(?! del contrato)(.*?)nif'
        matches_3 = re.findall(patron_cn_3, content, re.IGNORECASE)
        for match in matches_3:
            if match not in client_name:  # Solo agregar si no está en matches_1
                client_name.append(match)
        invoices["nombre_cliente"] = client_name[:]
        
        # ----------------------------------------------------------------------------------------------------------------------------
        # DNI   
        patron_nif = r'nif (\d{8}[a-zA-Z])'
        nif_1 = re.findall(patron_nif, content, re.IGNORECASE)
        for match in nif_1:
            if match not in dni:
                dni.extend(nif_1)
        invoices["dni_cliente"] = dni[:]

        #-----------------------------------------------------------------------------------------------------------------------------
        # Adress, city, province, cp
        patron_adress_1 = r"direccion\sde\ssuministro\s(?P<calle>.+?),\s(?P<localidad>.+?),\s(?P<provincia>.+?)\snumero\sde\scontador"
        adress_1 = re.search(patron_adress_1, content)
        if adress_1:
            if "calle_cliente" not in invoices:
                adress_street.append(adress_1.group("calle"))
                invoices["calle_cliente"] = adress_street
            if "población_cliente" not in invoices:
                adress_city.append(adress_1.group("localidad"))
                invoices["población_cliente"] = adress_city
            if "provincia_cliente" not in invoices:
                adress_province.append(adress_1.group("provincia"))
                invoices["provincia_cliente"] = adress_province

            # Patrón para encontrar el código postal entre la calle y la localidad
            if (n_pages >= 1):
                # Looking for cp from street
                found_street = adress_1.group("calle")
                found_city = adress_1.group("localidad")
                # Crear un patrón que busque el código postal entre la calle y la localidad
                patron_cp = fr'{re.escape(found_street)}\s+(\d{{5}})\s+{re.escape(found_city)}'
                cp_match = re.search(patron_cp, pages[n_pages - 1])

                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp
            if (invoices["nombre_cliente"]):
                found_client_name = invoices["nombre_cliente"][0]  # Suponiendo que solo hay un nombre almacenado en la lista
                # Patrón para encontrar el código postal después del nombre del cliente
                patron_cp = fr'{re.escape(found_client_name)}.*?(\d{{5}})'
                cp_match = re.search(patron_cp, pages[n_pages])
                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp

        patron_adress_2 = r"direccion\s+suministro\s+(?P<calle>.+?)\s+(?P<codigo_postal>\d{5})\s+(?P<localidad>.+?)\b"
        adress_2 = re.search(patron_adress_2, content)
        if adress_2:
            if "calle_cliente" not in invoices:
                adress_street.append(adress_2.group("calle"))
                invoices["calle_cliente"] = adress_street
            if "cp_cliente" not in invoices:
                adress_cp.append(adress_2.group("codigo_postal"))
                invoices["cp_cliente"] = adress_cp
            if "población_cliente" not in invoices:
                adress_city.append(adress_2.group("localidad"))
                invoices["población_cliente"] = adress_city
            # Patrón para encontrar el código postal entre la calle y la localidad
            if (n_pages >= 1):
                # Looking for cp from street
                found_street = adress_2.group("calle")
                found_city = adress_2.group("localidad")
                # Crear un patrón que busque el código postal entre la calle y la localidad
                patron_cp = fr'{re.escape(found_street)}\s+(\d{{5}})\s+{re.escape(found_city)}'
                cp_match = re.search(patron_cp, content)

                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp
            if (invoices["nombre_cliente"]):
                found_client_name = invoices["nombre_cliente"][0]
                patron_cp = fr'{re.escape(found_client_name)}.*?(\d{{5}})'
                cp_match = re.search(patron_cp, pages[n_pages])
                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp

        patron_adress = r"direccion de suministro\s+(?P<calle>.+?),\s+(?P<poblacion>.+?),\s+(?P<provincia>\w+)\s+numero de contador"
        adress_match = re.search(patron_adress, content)

        if adress_match:
            if "calle_cliente" not in invoices:
                adress_street.append(adress_match.group("calle"))
                invoices["calle_cliente"] = adress_street

            poblacion = adress_match.group("poblacion")
            if "población_cliente" not in invoices:
                invoices["población_cliente"] = [poblacion]
            elif len(poblacion) < len(invoices["población_cliente"][0]):
                invoices["población_cliente"] = [poblacion]

            if "provincia_cliente" not in invoices:
                adress_province.append(adress_match.group("provincia"))
                invoices["provincia_cliente"] = adress_province

        patron_direccion = r"calle\s+(?P<calle>.+?)\s+(?P<cp>\d{5})\s+(?P<localidad>.+?)\s+(?P<provincia>\w+)$"
        direccion_match = re.match(patron_direccion, content)

        if direccion_match:
            if "calle_cliente" not in invoices:
                adress_street.append(direccion_match.group("calle"))
                invoices["calle_cliente"] = adress_street

            poblacion = direccion_match.group("localidad")
            if "población_cliente" not in invoices:
                invoices["población_cliente"] = [poblacion]
                
            elif len(poblacion) < len(invoices["población_cliente"][0]):
                invoices["población_cliente"] = [poblacion]

            if "provincia_cliente" not in invoices:
                adress_province.append(direccion_match.group("provincia"))
                invoices["provincia_cliente"] = adress_province

        patron_adress_3 = r"calle\s+(?P<calle>.+?)\s+(\d{5})\s+(?P<localidad>.+?)\s+forma\s+de\s+pago\s+domiciliada"
        adress_3 = re.search(patron_adress_3, content)
        if adress_3:
            if "calle_cliente" not in invoices:
                adress_street.append("calle" + adress_3.group("calle"))
                invoices["calle_cliente"] = adress_street
            if "cp_cliente" not in invoices:
                adress_cp.append(adress_3.group(2))
                invoices["cp_cliente"] = adress_cp
            if "población_cliente" not in invoices:
                adress_city.append(adress_3.group("localidad"))
                invoices["población_cliente"] = adress_city
            # Patrón para encontrar el código postal entre la calle y la localidad
            if (n_pages >= 1):
                # Looking for cp from street
                found_street = adress_3.group("calle")
                found_city = adress_3.group("localidad")
                # Crear un patrón que busque el código postal entre la calle y la localidad
                patron_cp = fr'{re.escape(found_street)}\s+(\d{{5}})\s+{re.escape(found_city)}'
                cp_match = re.search(patron_cp, content)

                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp
            if (invoices["nombre_cliente"]):
                found_client_name = invoices["nombre_cliente"][0]  # Suponiendo que solo hay un nombre almacenado en la lista
                # Patrón para encontrar el código postal después del nombre del cliente
                patron_cp = fr'{re.escape(found_client_name)}.*?(\d{{5}})'
                cp_match = re.search(patron_cp, pages[n_pages])
                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp

        patron_adress_4 = r"direccion\sde\ssuministro\s(?P<calle>.+?)\s+(?P<codigo_postal>\d{5})\s+(?P<localidad>\w+)"
        adress_4 = re.search(patron_adress_4, content)
        if adress_4:
            if "calle_cliente" not in invoices:
                adress_street.append(adress_4.group("calle"))
                invoices["calle_cliente"] = adress_street
            if "cp_cliente" not in invoices:
                adress_cp.append(adress_4.group("codigo_postal"))
                invoices["cp_cliente"] = adress_cp
            if "población_cliente" not in invoices:
                adress_city.append(adress_4.group("localidad"))
                invoices["población_cliente"] = adress_city

            # Patrón para encontrar el código postal entre la calle y la localidad
            if (n_pages >= 1):
                # Looking for cp from street
                found_street = adress_4.group("calle")
                found_city = adress_4.group("localidad")
                # Crear un patrón que busque el código postal entre la calle y la localidad
                patron_cp = fr'{re.escape(found_street)}\s+(\d{{5}})\s+{re.escape(found_city)}'
                cp_match = re.search(patron_cp, pages[n_pages - 1])

                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp[0]
            if (invoices["nombre_cliente"]):
                found_client_name = invoices["nombre_cliente"][0]  # Suponiendo que solo hay un nombre almacenado en la lista
                # Patrón para encontrar el código postal después del nombre del cliente
                patron_cp = fr'{re.escape(found_client_name)}.*?(\d{{5}})'
                cp_match = re.search(patron_cp, pages[n_pages])
                if cp_match:
                    if "cp_cliente" not in invoices:
                        adress_cp.append(cp_match.group(1))
                        invoices["cp_cliente"] = adress_cp
        
        #----------------------------------------------------------------------------------------------
        # CIF de la empresa comercializadora
        patron_cif_1 = r"cif\s+(?P<cif>[a-zA-Z]\d{8})\."
        cif_match = re.search(patron_cif_1, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif
        # CIF de la empresa comercializadora
        patron_cif_2 = r"cif\s+(?P<cif>[a-zA-Z]\d{8})\s*\."
        cif_match = re.search(patron_cif_2, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif
        
        # CIF de la empresa comercializadora
        patron_cif_3 = r"cif\s+(?P<cif>[a-zA-Z]\d{8})\b"
        cif_match = re.search(patron_cif_3, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif

        # CIF de la empresa comercializadora
        patron_cif_4 = r"cif\s+(?P<cif>[a-zA-Z]{1,2}\d{6,8})\s*\."
        cif_match = re.search(patron_cif_4, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif

        # CIF de la empresa comercializadora
        patron_cif_5 = r"cif\s+(?P<cif>[a-zA-Z]{1,2}\d{6,8})\b"
        cif_match = re.search(patron_cif_5, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif

        patron_cif_6 = r"cif\s+(?P<cif>[a-zA-Z]{1,2}\d{6,9})\b"
        cif_match = re.search(patron_cif_6, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif

        # CIF de la empresa comercializadora
        patron_cif_1 = r"cif\s+(?P<cif>[a-zA-Z]{1,3}\s?\d{3}(\.?\d{3}){1,2}|\d{3}'\d{3}|\d{9,10})[\s\.]*"
        cif_match = re.search(patron_cif_1, pages[n_pages])

        if cif_match:
            if "cif_comercializadora" not in invoices:
                cif.append(cif_match.group("cif"))
                invoices["cif_comercializadora"] = cif

        #-----------------------------------------------------------------------------------------------
        # Dirección de la empresa 
        if (invoices['nombre_cliente']):
            if ("cif_comercializadora" in invoices):
                patron_location_1 = r"(?P<nombre_empresa>[^\d.]+)\s*\. cif\s*{}\s*\.\s*(?P<direccion>.*?)\s*(?P<cp>\d{{5}})\s*(?P<localidad>[^\s]*).*{}".format(invoices["cif_comercializadora"][0], re.escape(invoices["nombre_cliente"][0]))
                if (n_pages >= 1):
                    location_match = re.search(patron_location_1, pages[n_pages - 1], re.IGNORECASE)
                else:
                    location_match = re.search(patron_location_1, pages[n_pages], re.IGNORECASE)

                if location_match:
                
                    if "nombre_comercializadora" not in invoices:
                        distributor_name.append(location_match.group('nombre_empresa'))
                        invoices["nombre_comercializadora"] = distributor_name
    
                    if "dirección_comercializadora" not in invoices:
                        distributor_street.append(location_match.group('direccion').strip())
                        invoices["dirección_comercializadora"] = distributor_street
    
                    if "cp_comercializadora" not in invoices:
                        distributor_cp.append(location_match.group('cp'))
                        invoices["cp_comercializadora"] = distributor_cp
    
                    if "localidad_comercializadora" not in invoices:
                        distributor_city.append(location_match.group('localidad'))
                        invoices["localidad_comercializadora"] = distributor_city

                patron_location_2 = r"fecha de cargo.*?(?P<nombre_empresa>[^\d]+?)\s+cif\s+{}\s+(?P<direccion>.*?)\s+(?P<cp>\d{{5}})\s+(?P<localidad>[^\s]+)\s+{}".format(invoices["cif_comercializadora"][0], re.escape(invoices["nombre_cliente"][0]))
                if (n_pages >= 1):
                    location_match = re.search(patron_location_2, pages[n_pages - 1], re.IGNORECASE)
                else:
                    location_match = re.search(patron_location_2, pages[n_pages], re.IGNORECASE)

                if location_match:
                
                    if "nombre_comercializadora" not in invoices:
                        distributor_name.append(location_match.group('nombre_empresa'))
                        invoices["nombre_comercializadora"] = distributor_name
    
                    if "dirección_comercializadora" not in invoices:
                        distributor_street.append(location_match.group('direccion').strip())
                        invoices["dirección_comercializadora"] = distributor_street
    
                    if "cp_comercializadora" not in invoices:
                        distributor_cp.append(location_match.group('cp'))
                        invoices["cp_comercializadora"] = distributor_cp
    
                    if "localidad_comercializadora" not in invoices:
                        distributor_city.append(location_match.group('localidad'))
                        invoices["localidad_comercializadora"] = distributor_city

        patron_location_3 = r"(?<=mifactura\s)(?P<nombre_empresa>[A-Za-z\s,\.]+?)\s*\.?\s*dom\. social\s+(?P<direccion>.*?),\s*(?P<cp>\d{5})\s+(?P<localidad>[^\.\,]+)\.\s*(?P<provincia>[^\.\,]+)"
        location_match = re.search(patron_location_3, pages[n_pages], re.IGNORECASE)

        if location_match:
        
            if "nombre_comercializadora" not in invoices:
                distributor_name.append(location_match.group('nombre_empresa'))
                invoices["nombre_comercializadora"] = distributor_name

            if "dirección_comercializadora" not in invoices:
                distributor_street.append(location_match.group('direccion').strip())
                invoices["dirección_comercializadora"] = distributor_street

            if "cp_comercializadora" not in invoices:
                distributor_cp.append(location_match.group('cp'))
                invoices["cp_comercializadora"] = distributor_cp

            if "localidad_comercializadora" not in invoices:
                distributor_city.append(location_match.group('localidad'))
                invoices["localidad_comercializadora"] = distributor_city

            if "provincia_comercializadora" not in invoices:
                distributor_province.append(location_match.group('provincia'))
                invoices["provincia_comercializadora"] = distributor_province

        if ("cif_comercializadora" in invoices):
            patron_location_4 = r"(?P<nombre_empresa>[^\d]+?)\s+cif\s+{}\s+(?P<direccion>.*?)\s+(?P<cp>\d{{5}})\s+(?P<localidad>[^\s]+)".format(re.escape(invoices["cif_comercializadora"][0]))
            if (n_pages >= 1):
                location_match = re.search(patron_location_4, pages[n_pages - 1], re.IGNORECASE)
            else:
                location_match = re.search(patron_location_4, pages[n_pages], re.IGNORECASE)

            if location_match:
            
                if "nombre_comercializadora" not in invoices:
                    distributor_name.append(location_match.group('nombre_empresa'))
                    invoices["nombre_comercializadora"] = distributor_name

                if "dirección_comercializadora" not in invoices:
                    distributor_street.append(location_match.group('direccion').strip())
                    invoices["dirección_comercializadora"] = distributor_street

                if "cp_comercializadora" not in invoices:
                    distributor_cp.append(location_match.group('cp'))
                    invoices["cp_comercializadora"] = distributor_cp

                if "localidad_comercializadora" not in invoices:
                    distributor_city.append(location_match.group('localidad'))
                    invoices["localidad_comercializadora"] = distributor_city

        patron_location_5 =  r"\/mifactura\s+(?P<nombre_empresa>.*?)\.\s+dom\.?\s+social\s+(?P<direccion>.*?)\,\s+(?P<cp>\d{4})\s+(?P<localidad>[\w\s]+)\.\s+(?P<provincia>[\w\s]+),"
        location_match = re.search(patron_location_5, pages[n_pages], re.IGNORECASE)

        if location_match:
        
            if "nombre_comercializadora" not in invoices:
                distributor_name.append(location_match.group('nombre_empresa'))
                invoices["nombre_comercializadora"] = distributor_name

            if "dirección_comercializadora" not in invoices:
                distributor_street.append(location_match.group('direccion').strip())
                invoices["dirección_comercializadora"] = distributor_street

            if "provincia_comercializadora" not in invoices:
                distributor_province.append(location_match.group('provincia'))
                invoices["provincia_comercializadora"] = distributor_province
        
        patron_location_6 = r"domicilio\s+social\s+(?P<direccion>[^,]+),\s*(?P<localidad>[^\.,]+) xxxxx"
        location_match = re.search(patron_location_6, pages[n_pages])

        if location_match:
            
            if "dirección_comercializadora" not in invoices:
                distributor_street.append(location_match.group('direccion').strip())
                invoices["dirección_comercializadora"] = distributor_street
            if "localidad_comercializadora" not in invoices:
                distributor_city.append(location_match.group('localidad').strip())
                invoices["localidad_comercializadora"] = distributor_city

        # --------------------------------------------------------------------------------------------
        # Nombre de la comercializadora
        patron_name_1 = r"[\s.]+(?P<nombre_empresa>[^\.,]+)\.\s+inscrita"
        name_match = re.search(patron_name_1, pages[n_pages])

        if name_match:
            if "nombre_comercializadora" not in invoices:
                distributor_name.append(name_match.group('nombre_empresa'))
                invoices["nombre_comercializadora"] = distributor_name
          
    if (len(adress_province) == 0):
        adress_province = [encontrar_provincia_por_cp(invoices["cp_cliente"][0], "ES")]
        invoices["provincia_cliente"] = adress_province

    # Fill result
    result[name] = invoices

    # ------------------------------------------------------------------------------------------------------------------
    
    # Check dirección_comercializadora
    if len(distributor_street) == 0:
        empty["dirección_comercializadora"].append(name)
    else:
        filler["dirección_comercializadora"].append(name)

    # Check nombre_comercializadora
    if len(distributor_name) == 0:
        empty["nombre_comercializadora"].append(name)
    else:
        filler["nombre_comercializadora"].append(name)
    
    # Check cp_comercializadora
    if len(distributor_cp) == 0:
        empty["cp_comercializadora"].append(name)
    else:
        filler["cp_comercializadora"].append(name)
    
    # Check localidad_comercializadora
    if len(distributor_city) == 0:
        empty["localidad_comercializadora"].append(name)
    else:
        filler["localidad_comercializadora"].append(name)
    
    # Check distributor
    if len(cif) == 0:
        empty["cif_comercializadora"].append(name)
    else:
        filler["cif_comercializadora"].append(name)
    
    # Check cp_client
    if len(adress_cp) == 0:
        empty["cp_cliente"].append(name)
    else:
        filler["cp_cliente"].append(name)

    # Check calle_cliente
    if len(adress_street) == 0:
        empty["calle_cliente"].append(name)
    else:
        filler["calle_cliente"].append(name)

    # Check población_cliente
    if len(adress_city) == 0:
        empty["población_cliente"].append(name)
    else:
        filler["población_cliente"].append(name)

    # Check provincia_cliente
    if len(adress_province) == 0:
        empty["provincia_cliente"].append(name)
    else:
        filler["provincia_cliente"].append(name)

    # Check dni_cliente
    if len(dni) == 0:
        empty["dni_cliente"].append(name)
    else:
        filler["dni_cliente"].append(name)

    # Check nombre_cliente
    if len(client_name) == 0:
        empty["nombre_cliente"].append(name)
    else:
        filler["nombre_cliente"].append(name)
    
    return(result, empty, filler)

In [305]:
result, empty, filler = ft_font_code(clean_invoices[45], "Factura_44")

In [ ]:
# Código para todas las facturas

data_train = {}
for key, value in clean_invoices.items():
    data_train[f"Factura_{key}"] = {}
    result, empty, filler = ft_font_code(value, f"Factura_{key}")
    data_train[f"Factura_{key}"][f"Factura_{key}"] = result[f"Factura_{key}"]
    data_train[f"Factura_{key}"]["emptys"] = empty
    data_train[f"Factura_{key}"]["fillers"] = filler
    print(key)

In [317]:
diccionario = {}
for key, value in data_train.items():
    for apartado, dic in value.items():
        if apartado == key:
            if (len(dic) != 12):
                diccionario[key] = list(["nombre_cliente",
                            "dni_cliente",
                            "calle_cliente",
                            "cp_cliente",
                            "población_cliente",
                            "cif_comercializadora",
                            "nombre_comercializadora",
                            "dirección_comercializadora",
                            "cp_comercializadora",
                            "localidad_comercializadora",
                            "provincia_comercializadora",
                            "provincia_cliente"] - dic.keys())
    

In [323]:
provincia_comercializadora = 0
nombre_comercializadora = 0
cp_comercializadora = 0
localidad_comercializadora = 0
dirección_comercializadora = 0
nombre_cliente = 0
dni_cliente = 0
calle_cliente = 0
cp_cliente = 0
población_cliente = 0
cif_comercializadora = 0
provincia_cliente = 0

for i, n in diccionario.items():
    if 'nombre_cliente' in diccionario[i]:
        nombre_cliente += 1
    if 'dni_cliente' in diccionario[i]:
        dni_cliente += 1
    if 'calle_cliente' in diccionario[i]:
        calle_cliente += 1
    if 'cp_cliente' in diccionario[i]:
        cp_cliente += 1
    if 'población_cliente' in diccionario[i]:
        población_cliente += 1
    if 'provincia_cliente' in diccionario[i]:
        provincia_cliente += 1
    
    if 'nombre_comercializadora' in diccionario[i]:
        nombre_comercializadora += 1
    if 'cif_comercializadora' in diccionario[i]:
        cif_comercializadora += 1
    if 'dirección_comercializadora' in diccionario[i]:
        dirección_comercializadora += 1
    if 'cp_comercializadora' in diccionario[i]:
        cp_comercializadora += 1
    if 'localidad_comercializadora' in diccionario[i]:
        localidad_comercializadora += 1
    if 'provincia_comercializadora' in diccionario[i]:
        provincia_comercializadora += 1
    
print("nombre_cliente", nombre_cliente)
print("dni_cliente", dni_cliente)
print("calle_cliente", calle_cliente)
print("cp_cliente", cp_cliente)
print("población_cliente", población_cliente)
print("provincia_cliente", provincia_cliente)
print("nombre_comercializadora", nombre_comercializadora)
print("cif_comercializadora", cif_comercializadora)
print("dirección_comercializadora", dirección_comercializadora)
print("cp_comercializadora", cp_comercializadora)
print("localidad_comercializadora", localidad_comercializadora)
print("provincia_comercializadora", provincia_comercializadora)

nombre_cliente 0
dni_cliente 0
calle_cliente 0
cp_cliente 0
población_cliente 0
provincia_cliente 0
nombre_comercializadora 36
cif_comercializadora 1
dirección_comercializadora 40
cp_comercializadora 48
localidad_comercializadora 46
provincia_comercializadora 42


# Extracción:

- Nombre del cliente (nombre_cliente) **HECHO**
- DNI del cliente (dni_cliente) **HECHO**
- Calle del clilente (calle_cliente) **HECHO**
- Código postal del cliente (cp_cliente) **HECHO**
- Población del cliente (población_cliente) **HECHO**
- Provincia del cliente (provincia_cliente) **HECHO**
- Nombre de la empresa comercializadora (nombre_comercializadora) **HECHO MENOS 36 FACTURAS**
- CIF de la comercializadora (cif_comercializadora) **HECHO MENOS FACTURA 631**
- Dirección de la comercializadora (dirección_comercializadora) **HECHO MENOS 40 FACTURAS**
- Código postal de la comercializadora (cp_comercializadora) **HECHO MENOS 48 FACTURAS**
- Población de la comercializadora (población_comercializadora) **HECHO MENOS 46 FACTURAS**
- Provincia de la comercializadora (provincia_comercializadora) **HECHO 42 FACTURAS**
- Número de factura (nümero_factura)
- Inicio del periodo de facturación (inicio_periodo)
- Fin del periodo de facturacón (fin_periodo)
- Importe de la factura (importe_factura)
- Fecha del cargo (fecha_cargo)
- Consumo en el periodo (consumo_periodo)
- Potencia contratada (potencia_contratada)

In [248]:
# Comprobar vacíos
for key, value in result.items():
    for key_2, value_2 in  value.items():
        if (len(value[key_2]) == 0):
            print(key, key_2, len(value[key_2]))

In [249]:
# Comprobar faltantes
# Lista de claves que se esperan en el diccionario
expected_keys = ['dni_cliente', 'nombre_cliente', 'calle_cliente', 'población_cliente', 'provincia_cliente', 'cp_cliente', 'cif_comercializadora', 'nombre_comercializadora', 'dirección_comercializadora', 'localidad_comercializadora', 'cp_comercializadora', 'provincia_comercializadora']
dic_fails = {}

# Iterar sobre las claves y valores del diccionario resultante
for factura, datos in result.items():
    missing_keys = []
     # Verificar si el valor asociado tiene menos de 6 claves
    if len(datos) < len(expected_keys):
        # Encontrar las claves faltantes
        for key in expected_keys:
            if key not in datos:
                missing_keys.append(key)
    # Añadir las claves faltantes al diccionario
    if missing_keys:
        dic_fails[factura] = missing_keys

In [251]:
count = 0
for i in dic_fails.keys():
    if ("cp_comercializadora" in dic_fails[i]):
        count += 1
        print(i)
print(count)

Factura_20
Factura_27
Factura_72
Factura_109
Factura_124
Factura_143
Factura_148
Factura_251
Factura_283
Factura_292
Factura_298
Factura_367
Factura_377
Factura_383
Factura_386
Factura_395
Factura_413
Factura_426
Factura_436
Factura_444
Factura_504
Factura_518
Factura_521
Factura_527
Factura_545
Factura_555
Factura_561
Factura_588
Factura_594
Factura_596
Factura_605
Factura_631
Factura_647
Factura_651
Factura_711
Factura_713
Factura_742
Factura_765
Factura_780
Factura_789
Factura_796
Factura_850
Factura_860
Factura_863
Factura_908
Factura_931
Factura_937
Factura_962
48


In [6]:
# Función ft_client_name()

def ft_get_client_name(name):
    with open(f'{name}_clean.pkl', 'rb') as file:
        clean_invoices = pickle.load(file)
    result = {}
    empty = []
    filler = []

    for invoice, pages in clean_invoices.items():
        print(f"Factura_{invoice}")
        invoices = {}
        client_name = []
        for n_pages, content in pages.items():

            patron_1 = r'nombre (.*?) direccion'
            matches_1 = re.findall(patron_1, content, re.IGNORECASE)
            for match in matches_1:
                if match:
                    client_name.extend(matches_1)
            
            patron_2 = r'titular del contrato (.*?) nif'
            matches_2 = re.findall(patron_2, content, re.IGNORECASE)
            for match in matches_2:
                if match not in matches_1:  # Solo agregar si no está en matches_1
                    client_name.append(match)
            patron_3 = r'titular(?! del contrato)(.*?)nif'
            matches_3 = re.findall(patron_3, content, re.IGNORECASE)
            for match in matches_3:
                if match not in matches_1:  # Solo agregar si no está en matches_1
                    client_name.append(match)
            invoices["nombre_cliente"] = client_name[:]
            result[f"Factura_{invoice}"] = invoices
        if (len(client_name) == 1) & (client_name[0]== ""):
            name = f"Factura_{invoice}"
            empty.append(name)
        else:
            name = f"Factura_{invoice}"
            filler.append(name)
            
    return result

------------------------------------------------------------------------------------------------------------------